In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import re


# Configuração da conexão PostgreSQL
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "user": "your_username",
    "password": "your_password",
    "database": "your_database"
}

def get_postgres_engine(config):
    """Cria a conexão com o PostgreSQL usando SQLAlchemy."""
    url = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"
    return create_engine(url)


# Custom SQL function examples
def regexp_split_to_table(column, delimiter):
    return column.str.split(delimiter, expand=True).stack().reset_index(drop=True)

def cluster_function(data):
    # Dummy clustering function
    data["cluster"] = (data.index % 2) + 1
    return data


# SQLProcessor Class
class SQLProcessor:
    def __init__(self, db_engine):
        self.db_engine = db_engine
        self.temp_results = {}

    def execute(self, commands):
        """Execute a list of SQL commands."""
        for command in commands:
            self._process_command(command)

    def _process_command(self, command):
        parts = command.split(" ", 2)
        key = parts[0]
        action = parts[1]
        details = parts[2].strip() if len(parts) > 2 else ""

        if action == "from":
            self.temp_results[key] = self._process_from(details)
        elif action == "where":
            self.temp_results[key] = self._process_where(key, details)
        elif action == "select":
            self.temp_results[key] = self._process_select(key, details)
        elif action == "join":
            self.temp_results[key] = self._process_join(key, details)
        elif action == "group_by":
            self.temp_results[key] = self._process_group_by(key, details)
        elif action == "with":
            self._process_with(details)
        elif action == "function":
            self.temp_results[key] = self._process_function(key, details)
        else:
            raise ValueError(f"Unknown action: {action}")

    def _process_with(self, details):
        """Process CTEs (WITH statements)."""
        cte_name, cte_query = details.split(" AS ", 1)
        cte_name = cte_name.strip()
        self._process_command(cte_query.strip())
        self.temp_results[cte_name] = self.temp_results.pop(cte_query.strip())

    def _process_from(self, details):
        tables = [t.strip() for t in details.split(",")]
        dfs = []
        for table in tables:
            query = f"SELECT * FROM {table}"
            df = pd.read_sql_query(query, self.db_engine)
            dfs.append(df)
        # Merge tables if more than one is specified
        if len(dfs) > 1:
            df = dfs[0]
            for other_df in dfs[1:]:
                df = df.merge(other_df)
            return df
        return dfs[0]

    def _process_where(self, key, details):
        df = self.temp_results[key]
        query = details.replace("=", "==").replace("AND", "&").replace("OR", "|")
        query = re.sub(r"([\w_]+)\.([\w_]+)", r"df['\2']", query)  # Convert table.column to df['column']
        return df.query(query)

    def _process_select(self, key, details):
        df = self.temp_results[key]
        columns = [col.strip() for col in re.findall(r"\[([\w_]+)\]", details)]
        return df[columns]

    def _process_join(self, key, details):
        df = self.temp_results[key]
        join_type, other_table, on_condition = re.match(r"([\w_]+) JOIN ([\w_]+) ON (.+)", details).groups()
        right_df = pd.read_sql_query(f"SELECT * FROM {other_table}", self.db_engine)
        on_conditions = [cond.split("=") for cond in on_condition.split(" AND ")]
        on = [(left.strip(), right.strip()) for left, right in on_conditions]
        left_on, right_on = zip(*on)
        return df.merge(right_df, left_on=list(left_on), right_on=list(right_on), how=join_type.lower())

    def _process_group_by(self, key, details):
        df = self.temp_results[key]
        group_by_col = re.search(r"group_by ([\w_]+)", details).group(1)
        return df.groupby(group_by_col).size().reset_index(name="count")

    def _process_function(self, key, details):
        df = self.temp_results[key]
        if "REGEXP_SPLIT_TO_TABLE" in details:
            match = re.search(r"REGEXP_SPLIT_TO_TABLE\(([\w_]+), '([\w\s]*)'\)", details)
            column, delimiter = match.groups()
            split_col = regexp_split_to_table(df[column], delimiter)
            df = split_col.to_frame(name="word_tokenized").reset_index(drop=True)
        elif "cluster" in details:
            df = cluster_function(df)
        return df

    def get_result(self, key):
        return self.temp_results[key]


# PostgreSQL Connection
engine = get_postgres_engine(DB_CONFIG)

# Commands
commands = [
    "FIND_EVENT from event , date , description",
    "FIND_EVENT where fullDate == '28/7/2017' AND felling == 1 AND event.dateID == date.dataID AND event.descriptionID == description.descriptionID",
    "FIND_EVENT select title [ word ]",
    "COUNT_WORDS from FIND_EVENT",
    "COUNT_WORDS select REGEXP_SPLIT_TO_TABLE( word , ' ' ) [ word_tokenized ] , COUNT( * ) [ number ]",
    "COUNT_WORDS function REGEXP_SPLIT_TO_TABLE( word , ' ' )",
    "COUNT_WORDS group_by word_tokenized"
]

# Execute
processor = SQLProcessor(engine)
processor.execute(commands)

# Get final result
result = processor.get_result("COUNT_WORDS")
print(result)
